In [13]:
import sys
sys.path.append('..')
from envs import *
 
import matplotlib.pyplot as plt
import numpy as np
import pydirectinput
import pygame
from pygame.locals import *
import numpy as np
import os

from agents import DQN_agent
from agents.DQN_agent import *
import matplotlib.pyplot as plt

In [14]:
def simulate_key_press(key):
    pygame.event.post(pygame.event.Event(KEYDOWN, {'key': key}))

def perform_action(action):
    if action==0:
        simulate_key_press(K_UP)
    if action==1:
        simulate_key_press(K_RIGHT)
    if action==2:
        simulate_key_press(K_DOWN)
    if action==3:
        simulate_key_press(K_LEFT)
    if action==5:
        simulate_key_press(K_SPACE)

In [15]:
# Initialisation agent
input_shape = (12, 22, 22)  # pour 4 images RGB 22x22
num_actions = 6
agent = DQNAgent(input_shape=input_shape, num_actions=num_actions)
agent.model.load_state_dict(torch.load("C:/Users/DELL/Documents/Reinfrocement learning project/notebooks/checkpoints/model_ep2000.pth")
)

<All keys matched successfully>

In [16]:
#Evaluation mode i.e without exploration or training

agent.model.eval()  # Put the network on evaluation mode (no dropout, batchnorm, etc.)

env = TankEnv()
env.reset()
frame_stack = deque(maxlen=4)

# Set up the display
screen_size = (env.max_x * 20, env.max_y * 20)  # Scale up the game screen
screen = pygame.display.set_mode(screen_size)
clock = pygame.time.Clock()

current_frame=env.render() #The state now is the frame !

for _ in range(4):
    frame_stack.append(current_frame)
frame_state = np.concatenate(list(frame_stack), axis=2)

done=False
total_reward=0

while not done:
    state_tensor = torch.tensor(frame_state, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
    with torch.no_grad():
        q_values = agent.model(state_tensor)
        action = torch.argmax(q_values).item()
        
    #perform action
    perform_action(action)

    # Get the reward
    next_state, reward,kill_streak, done,_ = env.step(action)

    next_frame = env.render()
    frame_stack.append(next_frame)
    next_frame_state = np.concatenate(list(frame_stack), axis=2)

    #render the game
    frame=next_frame.copy()
    frame = np.repeat(np.repeat(frame, 18, axis=0), 18, axis=1)  # Scale up the frame for visibility
    surface = pygame.surfarray.make_surface(frame)
    screen.blit(surface, (0, 0))
    pygame.display.flip()

    # Cap the frame rate
    clock.tick(9)
    
    frame_state=next_frame_state
    total_reward += reward

print(f"[EVAL] Total reward : {total_reward}")
print(f"[EVAL] Final kill streak : {kill_streak}")

#### environnement reset successfully ####
[EVAL] Total reward : 4.980000000000056
[EVAL] Final kill streak : 9
